In [43]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import requests
from pandasql import sqldf
import json
from collections import defaultdict

In [44]:
print("Which Contest would you like to view?")
contest = input()
page = "https://codeforces.com/api/contest.status?contestId=" + contest + "&from=1"
cf_submissions_api = requests.get(page)
submissions = cf_submissions_api.json()
cf_dataframe = pd.json_normalize(submissions,['result'])

Which Contest would you like to view?


In [45]:
#fetch the duration of the contest
cf_duration_api = requests.get("https://codeforces.com/api/contest.list?gym=false")
contests = cf_duration_api.json()
cf_duration_dataframe = pd.json_normalize(contests,['result'])

selected_contest = cf_duration_dataframe.loc[cf_duration_dataframe["id"] == int(contest)]
start_time = list(selected_contest["startTimeSeconds"])[0]
end_time = list(selected_contest["startTimeSeconds"])[0] + list(selected_contest["durationSeconds"])[0] - 1

In [46]:
byTimeMemory = cf_dataframe[["creationTimeSeconds","contestId","problem.index","problem.name","programmingLanguage","author.members","verdict","timeConsumedMillis", "memoryConsumedBytes"]].loc[(cf_dataframe["creationTimeSeconds"] >= start_time) & (cf_dataframe["creationTimeSeconds"] <= end_time)]
byTimeMemory = byTimeMemory.rename(columns = {"timeConsumedMillis": "Time","problem.index":"problem_index","problem.name":"problem_name","author.members":"author_members"})

In [47]:
#retrieve the name and id of the problems
problem_header = byTimeMemory[["problem_index","problem_name"]]
problems = sorted(set(list(byTimeMemory["problem_index"])))
index_to_name = defaultdict()
for index,name in problem_header.values:
    index_to_name[index] = name
    if len(index_to_name) == len(problems):
        break

Acceptance rate per question (in leetcode format)

In [48]:
for pindex in problems:
    solved_df = byTimeMemory[(byTimeMemory["problem_index"] == pindex) & (byTimeMemory["verdict"] == "OK")]
    attempted_df = byTimeMemory[(byTimeMemory["problem_index"] == pindex)]
    user_solved = len(set([x[0]["handle"] for x in list(solved_df["author_members"])]))
    user_attempted = len(set([x[0]["handle"] for x in list(attempted_df["author_members"])]))
    total_solved = len(solved_df)
    total_attempted = len(attempted_df)
    acceptance_rate = ((total_solved / total_attempted) * 100)
    print(f"{contest}{pindex}. {index_to_name[pindex]}")
    print(f"User Accepted: {user_solved}")
    print(f"User Tried: {user_attempted}")
    print(f"Total Accepted: {total_solved}")
    print(f"Total Submissions: {total_attempted}")
    print(f"Acceptance Rate: {acceptance_rate} % ")
    print("-----------------------------------------------------------------------------------")

1672A. Log Chopping
User Accepted: 7432
User Tried: 8209
Total Accepted: 7432
Total Submissions: 12581
Acceptance Rate: 59.07320562753359 % 
-----------------------------------------------------------------------------------
1672B. I love AAAB
User Accepted: 6768
User Tried: 8108
Total Accepted: 6768
Total Submissions: 24533
Acceptance Rate: 27.58733134961073 % 
-----------------------------------------------------------------------------------
1672C. Unequal Array
User Accepted: 5194
User Tried: 6174
Total Accepted: 5194
Total Submissions: 11397
Acceptance Rate: 45.57339650785294 % 
-----------------------------------------------------------------------------------
1672D. Cyclic Rotation
User Accepted: 1871
User Tried: 3173
Total Accepted: 1871
Total Submissions: 8682
Acceptance Rate: 21.550334024418337 % 
-----------------------------------------------------------------------------------
1672E. notepad.exe
User Accepted: 564
User Tried: 1152
Total Accepted: 564
Total Submissions: 471